``` PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res:  <http://dbpedia.org/resource/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT DISTINCT ?uri, ?name, ?birth, ?position, ?number
WHERE { 
        {?uri dbo:team res:Boston_Celtics} UNION {?uri dbo:draftTeam res:Boston_Celtics}
        ?uri foaf:name ?name .
        ?uri dbo:birthDate ?birth .
        FILTER (regex (?birth, '\\d')) .
        ?uri dbo:position ?position .
        ?uri dbo:number ?number
} 
```

In [400]:
import pandas as pd
import spacy
import re
from datetime import datetime
import calendar
from collections import defaultdict
import wikipediaapi
import nltk
from nltk.metrics import edit_distance 
from word2number import w2n
pd.options.mode.chained_assignment = None
import numpy as np
# nlp = spacy.load("en_core_web_md")
# nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [32]:
df = pd.read_csv('/Users/liudmylaslava/Downloads/sparql.csv')
df.head()

,uri,name,birth,position,number
0,http://dbpedia.org/resource/James_Young_(baske...,James Young,1995-8-16,http://dbpedia.org/resource/Shooting_guard,13
1,http://dbpedia.org/resource/James_Young_(baske...,James Young,1995-8-16,http://dbpedia.org/resource/Small_forward,13
2,http://dbpedia.org/resource/Isaiah_Thomas_(bas...,Isaiah Thomas,1989-2-7,http://dbpedia.org/resource/Point_guard,4
3,http://dbpedia.org/resource/Jae_Crowder,Jae Crowder,1990-7-6,http://dbpedia.org/resource/Small_forward,99
4,http://dbpedia.org/resource/Jonas_Jerebko,Jonas Jerebko,1987-3-2,http://dbpedia.org/resource/Forward_(basketball),8


In [196]:
df['uri'] = df['uri'].apply(lambda x:x.split('/')[-1])
df['position'] = df['position'].apply(lambda x:x.split('/')[-1].replace('_',' '))
df.head()

,uri,name,birth,position,number
0,James_Young_(basketball),James Young,1995-8-16,Shooting guard,13
1,James_Young_(basketball),James Young,1995-8-16,Small forward,13
2,Isaiah_Thomas_(basketball),Isaiah Thomas,1989-2-7,Point guard,4
3,Jae_Crowder,Jae Crowder,1990-7-6,Small forward,99
4,Jonas_Jerebko,Jonas Jerebko,1987-3-2,Forward (basketball),8


In [346]:
w = wikipediaapi.Wikipedia('en')
ents = list(set(df['uri'].tolist()))

text = defaultdict(list)
for ent in ents:
    page = w.page(ent)
    text[ent] = page.text

In [347]:
with open('text.txt', 'w') as f:
    f.writelines(text)

In [372]:
positions = [ 'point guard' , 'shooting guard', 'small forward', 'power forward', 'center']
data = []
for k, t in text.items():
    name, birth, position, number = 0,0,0,0
    doc = nlp(t)
    sentences = [sent.string.strip() for sent in doc.sents]
    first = ''.join(sentences[:3])
    birth = first[first.find("(")+1:first.find(")")]
    name = first[0:first.find('(')]
    po = []
    for pos in positions:
        if pos in t:
            po.append(pos)
            position = ','.join(po)
    num = []
    for i, token in enumerate(doc):
        if token.text == 'No':
            if doc[i+1].text == '.':
                num.append(doc[i+2].text)
        elif token.text == 'number':
            if doc[i+1].pos_=='NUM':
                n = w2n.word_to_num(doc[i+1].text)
                num.append(str(n))
        number = ','.join(num)
                 
    data.append([k, name, birth, position, number])

In [376]:
raw = pd.DataFrame(data = data, columns = ['uri','name', 'birth', 'position', 'number'])
raw.head()

,uri,name,birth,position,number
0,Bulbs_Ehlers,"Edwin S. ""Bulbs"" Ehlers","March 10, 1923 – June 17, 2013",0,
1,Orien_Greene,Orien Randolph Greene II,"born February 4, 1982",point guard,
2,John_Richter,John Fritz Richter,"March 12, 1937 – March, 1983",0,
3,Jon_Barry,Jon Alan Barry,"born July 25, 1969",0,
4,Jim_Barnett_(basketball),James Franklin Barnett,"born July 7, 1944",0,


In [377]:
m ={}
for i in range(1,13):
    m[i] = calendar.month_name[i]
list(m.keys())[list(m.values()).index('December')]
raw['name'] = raw['name'].apply(lambda x: x.replace(',',''))
raw['birth'] = raw['birth'].apply(lambda x: x.split('–')[0].replace(',',''))
raw['birth'] = raw['birth'].apply(lambda x: x.split('-')[0].replace(',',''))
raw['birth'] = raw['birth'].apply(lambda x: ', '.join([t.text for t in nlp(x) if t.ent_type_=='DATE']))
raw['year']= raw['birth'].str[-4:]
raw['month'] = raw['birth'].apply(lambda x: ''.join([t.text for t in nlp(x) if t.pos_=='PROPN']))
raw['bir'] = raw['birth'].str[:-6]
raw['day'] = raw['bir'].apply(lambda x: ''.join([t.text for t in nlp(x) if t.pos_=='NUM']))
raw['month'] = raw[raw['month']!='']['month'].apply(lambda x: list(m.keys())[list(m.values()).index(x)])
raw['month'] = raw['month'].astype(str).str[:1]
raw['birth'] = raw['year']+'-'+raw['month']+'-'+raw['day']
raw.drop(['year','month','bir','day'], axis=1, inplace=True)
raw.head()

,uri,name,birth,position,number
0,Bulbs_Ehlers,"Edwin S. ""Bulbs"" Ehlers",1923-3-10,0,
1,Orien_Greene,Orien Randolph Greene II,1982-2-4,point guard,
2,John_Richter,John Fritz Richter,1937-3-12,0,
3,Jon_Barry,Jon Alan Barry,1969-7-25,0,
4,Jim_Barnett_(basketball),James Franklin Barnett,1944-7-7,0,


# 1. Перевіримо схожість імен

In [378]:
res = pd.merge(raw, df, on='uri')
res = res[['name_x','name_y']].drop_duplicates()
res.head()

,name_x,name_y
0,"Edwin S. ""Bulbs"" Ehlers",Bulbs Ehlers
1,"Edwin S. ""Bulbs"" Ehlers","Edwin ""Bulbs"" Ehlers"
4,Orien Randolph Greene II,Orien Greene
5,John Fritz Richter,John Richter
6,Jon Alan Barry,Jon Barry


In [379]:
def f(x):
    res = any(z == y for z in x[0].split() for y in x[1].split())
    return res
res['match'] = res.apply(f, axis=1)
res['match'].value_counts() # хоча б одне слово у імені завжди співпадає, скоріше за все це -- прізвище

True    108
Name: match, dtype: int64

In [380]:
# Нормалізована відстань Левінштайна
res['norm_dist'] = res.apply(lambda x: (1-edit_distance(x["name_x"], x["name_y"])/max(len(x["name_x"]),len(x["name_y"]))), axis=1)
res['norm_dist'].mean()

0.6358794407330797

# 2. Як співпадає дата народження

In [381]:
res = pd.merge(raw, df, on='uri')
birth = res[['birth_x','birth_y']].drop_duplicates()
birth['match'] = birth['birth_x']==birth['birth_y']
birth['match'].value_counts()

True     96
False    10
Name: match, dtype: int64

In [382]:
precision = len(birth[birth['match']==True])/len(birth)
precision

0.9056603773584906

# 3. Як співпадає позиція гравця

In [397]:
pos_pred = raw[['uri','position']]
pos_pred.loc[:, 'position'] = pos_pred.position.str.split(',')
pos_pred['position'] = pos_pred.loc[pos_pred['position'].notnull()]['position'].apply(lambda x: [y.replace(' ','') for y in x])
pos_true = df.groupby('uri')['position'].apply(list).to_frame()
pos_true['position'] = pos_true['position'].apply(lambda x: [y.replace('(basketball)','') for y in x])
pos_true['position'] = pos_true['position'].apply(lambda x: [y.lower() for y in x])
pos_true['position'] = pos_true['position'].apply(lambda x: [y.replace(' ','') for y in x])

In [393]:
def f(x):
    res = sum(z == y for z in x[0] for y in x[1])
    return res
pos = pd.merge(pos_true, pos_pred, on='uri')
pos['match'] = pos[pos['position_y'].notnull()][['position_x', 'position_y']].apply(f,axis=1)
pos['match'] = pos['match'].fillna(0)
pos['length'] = pos['position_x'].apply(lambda x: len(x))
pos['res'] = pos['match']/pos['length']
pos['res'].mean() # 30% позицій вдалося витягти з тексту

0.3018867924528302

# 4. Як співпадає номер гравця

In [410]:
num_pred = raw[['uri', 'number']]
num_pred.loc[:, 'number'] = num_pred.number.str.split(',')
num_true = df.groupby('uri')['number'].apply(lambda x: list(np.unique(x))).to_frame()
num = pd.merge(num_pred, num_true, on='uri')
num['match'] = num[['number_x','number_y']].apply(f, axis=1)
num['match'].value_counts()

0    104
1      2
Name: match, dtype: int64

 Номер гравця вдалося дістати тільки у 2/106 випадках тому що :
 - номер змінюється в залежності від команди де він грає
 - слова 'Number' чи 'No.' використовуються для пояснення успіхів баскетболіста а не його номеру у команді
 - у тексті вікі цієї інформації зазвичай нема, вона є тільки у табличних даних
